In [1]:
import tensorflow as tf
import tensorflow.keras as keras


class conv_block(keras.Model):
    def __init__(self,filters,kernel_size = 3,strides = 1):
        super(conv_block, self).__init__()
        self.conv = keras.layers.Conv2D(
                            filters=filters,kernel_size=kernel_size,
                            strides = strides,padding='same',
                            use_bias=False)
        self.bn = keras.layers.BatchNormalization()
        self.relu = keras.layers.Activation('relu')

    def call(self,x):
        Y = self.relu(self.bn(self.conv(x)))
        return Y

In [2]:
class depthwise_conv_block(keras.Model):
    def __init__(self,pointwise_conv_filters,strides = 1):
        super(depthwise_conv_block, self).__init__()
        self.dwConv = keras.layers.DepthwiseConv2D(
                        kernel_size=(3,3),  strides=strides,
                        padding='SAME',use_bias=False)
        self.bn1 = keras.layers.BatchNormalization()
        self.relu1 = keras.layers.Activation('relu')

        self.conv1x1 = keras.layers.Conv2D(filters=pointwise_conv_filters,
                            kernel_size=(1,1),
                            padding='SAME', use_bias=False)
        self.bn2 = keras.layers.BatchNormalization()
        self.relu2 = keras.layers.Activation('relu')

    def call(self,x):
        Y = self.relu1(self.bn1(self.dwConv(x)))
        Y = self.relu2(self.bn2(self.conv1x1(Y)))
        return Y

In [8]:
def mobilenet_v1(num_classes = 1000):
    return keras.models.Sequential([
        conv_block(filters=32,strides=2),
        depthwise_conv_block(64),
        depthwise_conv_block(128,strides=2),
        depthwise_conv_block(128),
        depthwise_conv_block(256,strides=2),
        depthwise_conv_block(256),
        depthwise_conv_block(512,strides=2),
        depthwise_conv_block(512),
        depthwise_conv_block(512),
        depthwise_conv_block(512),
        depthwise_conv_block(512),
        depthwise_conv_block(512),
        depthwise_conv_block(1024,strides=2),
        depthwise_conv_block(1024),

        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(num_classes)
    ])

In [9]:
net = mobilenet_v1(1000)

X = tf.random.uniform(shape=(1, 224, 224, 3))
for layer in net.layers:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

conv_block output shape:	 (1, 112, 112, 32)
depthwise_conv_block output shape:	 (1, 112, 112, 64)
depthwise_conv_block output shape:	 (1, 56, 56, 128)
depthwise_conv_block output shape:	 (1, 56, 56, 128)
depthwise_conv_block output shape:	 (1, 28, 28, 256)
depthwise_conv_block output shape:	 (1, 28, 28, 256)
depthwise_conv_block output shape:	 (1, 14, 14, 512)
depthwise_conv_block output shape:	 (1, 14, 14, 512)
depthwise_conv_block output shape:	 (1, 14, 14, 512)
depthwise_conv_block output shape:	 (1, 14, 14, 512)
depthwise_conv_block output shape:	 (1, 14, 14, 512)
depthwise_conv_block output shape:	 (1, 14, 14, 512)
depthwise_conv_block output shape:	 (1, 7, 7, 1024)
depthwise_conv_block output shape:	 (1, 7, 7, 1024)
GlobalAveragePooling2D output shape:	 (1, 1024)
Dense output shape:	 (1, 1000)
